In [1]:
from bs4 import BeautifulSoup
import urllib
from datetime import datetime
from datetime import timedelta
import pandas as pd
import pickle
import numpy as np
import pandas as pd
import nltk
from sklearn.externals import joblib
import re
import codecs
from sklearn import feature_extraction
import os
from bs4 import BeautifulSoup
import urllib
from nltk.stem.snowball import SnowballStemmer
import scipy.spatial.distance
import numba 
from numba import jit


nyt_unique=pd.read_csv('nyt_uniques_topicsLabeled.csv')
reuters_unique = pickle.load( open( "20180516-20180621_reuters_unique.pkl", "rb" ) )
reuters_titles_first_10000 = pickle.load( open( 'reuters_titles_first10000.pkl', "rb" ) )
reuters_titles_rest = pickle.load( open( 'reuters_titles_rest.pkl', "rb" ) )

def token_stem(text):
    # tokenize by sentence and word. this way you ensure you get rid of punctuations
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    tokens_with_letters = []
    # use the regex library to search only for items that contain letters. this will enable you to eliminate punctuation
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            tokens_with_letters.append(token)
    stems = [stemmer.stem(t) for t in tokens_with_letters] #"stems" part
    return stems

def token(text): #difference between this one and the one above is the "stems" part
    #same as above
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    tokens_with_letters = []
    # same as above
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            tokens_with_letters.append(token)
    return tokens_with_letters

reuters_unique.reset_index(inplace=True)
reuters_unique.drop('index',axis=1,inplace=True)
reuters_titles_first_10000.extend(reuters_titles_rest)
reuters_unique['titles']=pd.Series(reuters_titles_first_10000)

nyt_labeled_only=nyt_unique.loc[nyt_unique['Section'] != 'Other']
nyt_labeled_only.reset_index(inplace=True)
nyt_labeled_only.drop('index',axis=1,inplace=True)

nyt_titles=nyt_labeled_only['Title'].tolist()
#now, keep in mind that we're adding reuters titles to nyt_titles, so we essentially have all the titles in a 
#variable called "nyt_titles". 
nyt_titles.extend(reuters_titles_first_10000)

for n,i in enumerate(nyt_titles):
    if type(i)!=str: 
        nyt_titles[n]="URL Error"

nyt_titles=list(map(lambda unicode_line:unicode_line.translate({ord(c): None for c in 'Äòôúîù'}),nyt_titles))

stopwords = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer("english") 

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english',use_idf=True, norm='l2',tokenizer=token_stem, ngram_range=(1,3))

tfidf_matrix = vectorizer.fit_transform(nyt_titles)

nyt_titles_tfidf=tfidf_matrix[0:1400,:].toarray()
reuters_titles_tfidf=tfidf_matrix[1400:,:].toarray()

print(tfidf_matrix.shape)


/home/adlimberkey/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


(23722, 234972)


In [2]:
#title_distances=scipy.spatial.distance.cdist(nyt_titles_tfidf,reuters_titles_tfidf,'cosine')
#pf=open('title_distances.pkl','wb')
#pickle.dump(title_distances,pf)
title_distances = pickle.load( open( "title_distances.pkl", "rb" ) )
title_distances.shape

(1400, 22322)

In [3]:
Columns=['NYT_Orig.Index','NYT_Date_Pub','NYT_Link','NYT_Keywords','NYT_Article','NYT_Title','NYT_Section',
        'NYT_Date_Printed','NYT_Missing_Articles/Day','Reuters_Date','Reuters_Link','Reuters_Keywords','Reuters_Article',
        'Reuters_Title','title_distance']



In [4]:
#change the repeat to title_distances.shape[1], nyt_x to nyt_labeled_only
flatten = lambda l: [item for sublist in l for item in sublist]
nyt_df=pd.DataFrame(np.repeat(nyt_labeled_only.values,title_distances.shape[1],axis=0),columns=nyt_labeled_only.columns)
reuters_df=pd.DataFrame(pd.np.tile(reuters_unique, (title_distances.shape[0], 1)),columns=reuters_unique.columns)
title_dist_series=pd.Series(flatten(title_distances.tolist()))
big_df=pd.concat([nyt_df, reuters_df], axis=1)
big_df['title_distances']=pd.Series(flatten(title_distances.tolist()))
big_df.columns=Columns
pf=open('big_df.pkl','wb')
pickle.dump(big_df,pf)

In [7]:
big_df.shape

(31250800, 15)

In [8]:
len(nyt_titles)

23722

In [11]:
length_list=[]
for i in nyt_titles:
    length_list.append(len(i.split()))
    
sum(length_list)/len(length_list)
    

9.962692858949499

In [9]:
big_df.shape

(31250800, 15)